In [65]:
from fastapi import FastAPI
import uvicorn
import pandas as pd
import json as json


dfMoviesFinal = pd.read_parquet('DataSets/dfMoviesFinal.parquet')
dfCreditsFinal = pd.read_parquet('DataSets/dfCreditsFinal.parquet')



In [68]:
# Eliminar filas donde 'id_pelicula' no es convertible a float
dfMoviesFinal = dfMoviesFinal[dfMoviesFinal['id_pelicula'].astype(str).str.replace('.', '', regex=False).str.isnumeric()]
dfCreditsFinal = dfCreditsFinal[dfCreditsFinal['id_pelicula'].astype(str).str.replace('.', '', regex=False).str.isnumeric()]

# Convertir la columna 'id_pelicula' a tipo float en ambos DataFrames
dfMoviesFinal['id_pelicula'] = dfMoviesFinal['id_pelicula'].astype(float)
dfCreditsFinal['id_pelicula'] = dfCreditsFinal['id_pelicula'].astype(float)



In [69]:
def cantidad_filmaciones_mes(mes: str):
    meses_dict = {
        'enero': 1, 'febrero': 2, 'marzo': 3, 'abril': 4, 'mayo': 5, 'junio': 6,
        'julio': 7, 'agosto': 8, 'septiembre': 9, 'octubre': 10, 'noviembre': 11, 'diciembre': 12
    }
    mes = mes.lower()
    if mes not in meses_dict:
        raise ValueError("Mes no válido. Debe ser uno de los siguientes: enero, febrero, marzo, abril, mayo, junio, julio, agosto, septiembre, octubre, noviembre, diciembre.")
    mes_numero = meses_dict[mes]
 
    dfMoviesFinal['mes'] = dfMoviesFinal['release_date'].dt.month
    df_mes = dfMoviesFinal[dfMoviesFinal['mes'] == mes_numero]
    cantidad = df_mes['id_pelicula'].nunique()
    
    return f"{cantidad} películas fueron estrenadas en el mes de {mes}"



In [9]:
def cantidad_filmaciones_dia(dia):

    dias_dict = {
        'lunes': 1, 'martes': 2, 'miercoles': 3, 'jueves': 4, 'viernes': 5, 'sabado': 6, 'domingo': 7
    }
    dia = dia.lower()

    if dia not in dias_dict:
        raise ValueError("Día no válido. Debe ser uno de los siguientes: lunes, martes, miércoles, jueves, viernes, sábado, domingo.")
    
    dia_numero = dias_dict[dia]

    dfMoviesFinal['dia_semana'] = dfMoviesFinal['release_date'].dt.dayofweek + 1

    cantidad = dfMoviesFinal[dfMoviesFinal['dia_semana'] == dia_numero]['id_pelicula'].nunique()
    
    return f"{cantidad} cantidad de películas fueron estrenadas en los días {dia}"

In [10]:
def score_titulo(nombre_peli: str):

    pelicula = dfMoviesFinal[dfMoviesFinal['title'].str.contains(nombre_peli, case=False, na=False)]

    if pelicula.empty:
        return f"No se encontró la película '{nombre_peli}'."

    titulo = pelicula.iloc[0]['title']
    anio_estreno = pelicula.iloc[0]['release_year']
    puntuacion = pelicula.iloc[0]['vote_average']

    return f"La película '{titulo}' fue estrenada en el año {anio_estreno} con un score/popularidad de {puntuacion}."


In [12]:
def votos_titulo(titulo_de_la_filmacion: str):
    pelicula = dfMoviesFinal[dfMoviesFinal['title'].str.contains(titulo_de_la_filmacion, case=False, na=False)]
    
    if pelicula.empty:
        return f"No se encontró la película '{titulo_de_la_filmacion}'."
    
    titulo = pelicula.iloc[0]['title']
    anio_estreno = pelicula.iloc[0]['release_year']
    cantidad_votos = pelicula.iloc[0]['vote_count']
    promedio_votos = pelicula.iloc[0]['vote_average']
    
    if cantidad_votos < 2000:
        return f"La película '{titulo}' no cumple con la condición de tener al menos 2000 valoraciones (tiene {cantidad_votos} valoraciones)."
    
    return (f"La película '{titulo}' fue estrenada en el año {anio_estreno}. "
            f"La misma cuenta con un total de {cantidad_votos} valoraciones, con un promedio de {promedio_votos}.")


In [59]:
def get_actor(nombre_actor: str):
    # Filtrar el DataFrame dfCreditsFinal para encontrar las películas en las que ha actuado el actor
    peliculas_actor = dfCreditsFinal[dfCreditsFinal['name_cast'].str.contains(nombre_actor, case=False, na=False)]

    # Verificar si el actor aparece en el dataset
    if peliculas_actor.empty:
        return f"No se encontró al actor {nombre_actor} en el dataset."

    # Obtener las películas únicas en las que ha participado, eliminando duplicados de 'id_pelicula'
    ids_peliculas = peliculas_actor['id_pelicula'].drop_duplicates()

    # Filtrar las películas en el DataFrame dfMoviesFinal usando los IDs obtenidos
    peliculas_info = dfMoviesFinal[dfMoviesFinal['id_pelicula'].isin(ids_peliculas)]
    peliculas_info = peliculas_info.drop_duplicates(subset='id_pelicula')

    # Calcular el retorno total y promedio
    retorno_total = peliculas_info['revenue'].sum()
    retorno_promedio = retorno_total/len(ids_peliculas)

    # Devolver la cantidad de películas y el retorno total y promedio
    cantidad_peliculas = len(ids_peliculas)
    if cantidad_peliculas == 0:
        return f"El actor {nombre_actor} no tiene películas con presupuesto válido."

    return (f"El actor {nombre_actor} ha participado en {cantidad_peliculas} películas, "
            f"con un retorno total de {retorno_total:.2f} y un promedio de {retorno_promedio:.2f} por filmación.")


In [98]:
def get_director(nombre_director: str):
    # Filtrar el DataFrame dfCreditsFinal para encontrar las películas dirigidas por el director
    peliculas_director = dfCreditsFinal[
        (dfCreditsFinal['name_crew'].str.contains(nombre_director, case=False, na=False)) &
        (dfCreditsFinal['job_crew'].str.contains('Director', case=False, na=False))
    ]
    
    # Verificar si el director tiene películas en el dataset
    if peliculas_director.empty:
        return f"El director {nombre_director} no se encuentra en el dataset."

    # Obtener los IDs de las películas dirigidas por el director
    ids_peliculas = peliculas_director['id_pelicula'].unique()
    
    # Filtrar el DataFrame dfMoviesFinal usando los IDs de las películas del director
    peliculas_info = dfMoviesFinal[dfMoviesFinal['id_pelicula'].isin(ids_peliculas)]
    peliculas_info = peliculas_info.drop_duplicates(subset='id_pelicula')

    # Calcular el retorno y extraer la información de las películas
    peliculas_info['ganancia'] = peliculas_info['revenue'] - peliculas_info['budget']
    peliculas_info['retorno'] = peliculas_info['revenue']

    # Preparar la salida
    resultados = []
    for _, row in peliculas_info.iterrows():
        resultado = (
            f"Película: {row['title']}, "
            f"Fecha de lanzamiento: {row['release_date']}, "
            f"Retorno: {row['retorno']:.2f}, "
            f"Costo: {row['budget']:.2f}, "
            f"Ganancia: {row['ganancia']:.2f}"
        )
        resultados.append(resultado)
    
    return "\n".join(resultados)


In [99]:
get_director('stanley kubrick')

"Película: Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb, Fecha de lanzamiento: 1964-01-29, Retorno: 9440272.00, Costo: 1800000.00, Ganancia: 7640272.00\nPelícula: 2001: A Space Odyssey, Fecha de lanzamiento: 1968-04-10, Retorno: 68700000.00, Costo: 10500000.00, Ganancia: 58200000.00\nPelícula: Paths of Glory, Fecha de lanzamiento: 1957-09-18, Retorno: 0.00, Costo: 935000.00, Ganancia: -935000.00\nPelícula: A Clockwork Orange, Fecha de lanzamiento: 1971-12-18, Retorno: 26589000.00, Costo: 2200000.00, Ganancia: 24389000.00\nPelícula: Full Metal Jacket, Fecha de lanzamiento: 1987-06-26, Retorno: 46357676.00, Costo: 17000000.00, Ganancia: 29357676.00\nPelícula: The Shining, Fecha de lanzamiento: 1980-05-22, Retorno: 44017374.00, Costo: 19000000.00, Ganancia: 25017374.00\nPelícula: Eyes Wide Shut, Fecha de lanzamiento: 1999-07-14, Retorno: 162091208.00, Costo: 65000000.00, Ganancia: 97091208.00\nPelícula: The Killing, Fecha de lanzamiento: 1956-05-08, Retorno: 0.00, 